# Microsimulation with artificial survey data 

In [ ]:
import matplotlib.pyplot as plt  # For graphics
%matplotlib inline

import numpy as np  # linear algebra and math
import pandas as pd  # data frames

from openfisca_core.model_api import *
from openfisca_senegal import CountryTaxBenefitSystem as SenegalTaxBenefitSystem  # The Senegalese tax-benefits system

from openfisca_senegal.survey_scenarios import SenegalSurveyScenario

## Building the artificial data

Sénégal is composed by almost 15 millions people dispatched in around 1.6 million household

In [ ]:
household_weight = 100
size = int(1.6e6 / household_weight)
print("Size of the sample: {}".format(size))
np.random.seed(seed = 42)

We assume that 2/3 of the household heads are married and that only married houshold do have children. The mean number of children per household is 5 and is normally distributed

In [ ]:
est_marie = np.random.binomial(1, .66, size = size)
est_celibataire = np.logical_not(est_marie)
nombre_enfants = np.maximum(
    est_marie * np.round(np.random.normal(5, scale = 3, size = size)),
    0,
    )

We assume that 80% of the population are wage earners.
We choose a mean wage of 5 000 0000 CFA with a log normal ditribution.
Since 
$$ \text{mean wage}  = e^{\mu + \frac{\sigma ^ 2}{2}} $$ 
and
$$ \text{median wage} = e^\mu $$
we can compute the distribution according to the following expressions.

In [ ]:
mean_wage = 5e6
median_wage = .75 * mean_wage
est_salarie = np.random.binomial(1, .8, size = size)
mu = np.log(median_wage)
sigma = np.sqrt(2 * np.log(mean_wage / median_wage))
salaire = (
    est_salarie * 
    np.random.lognormal(mean = mu, sigma = sigma, size = int(size))
    )


We choose a mean pension of 2 500 000 CFA

In [ ]:
mean_pension = 2.5e6
median_pension = .9 * mean_pension

In [ ]:
mu = np.log(median_pension)
sigma = np.sqrt(2 * np.log(mean_pension / median_pension))
pension_retraite = (
    np.logical_not(est_salarie) *
    np.random.lognormal(mean = mu, sigma = sigma, size = int(size))
    )

In [ ]:
input_data_frame = pd.DataFrame({
    'est_marie': est_marie,
    'est_celibataire': est_celibataire,
    'nombre_enfants': nombre_enfants,
    'pension_retraite': pension_retraite,
    'salaire': salaire,
    'household_id': range(size),
    'household_role_index': 0,
    })

In [ ]:
input_data_frame.salaire.hist(bins=100)

In [ ]:
input_data_frame.pension_retraite.hist(bins=100, range = [.0001, 1e8])

## Microsimulation 

As with test case, we can build a scenario with survey data

In [ ]:
data = dict(input_data_frame = input_data_frame)
scenario = SenegalSurveyScenario(
    data = data,
    year = 2017,
    )

We can compute the value of any variable for the whole population an draw distributions

In [ ]:
scenario.simulation.calculate('impot_revenus', period = 2017)

In [ ]:
df = pd.DataFrame({'impot_revenus': scenario.simulation.calculate('impot_revenus', period = 2017)})

In [ ]:
df.hist(bins = 100)

Special methods allow access to aggregates and pivot tables

In [ ]:
scenario.compute_aggregate('impot_revenus', period = 2017) / 1e9

In [ ]:
scenario.compute_pivot_table(
    aggfunc = 'sum', 
    values = ['impot_revenus'], 
    columns = ['nombre_enfants'],
    period = 2017,
    ).stack().reset_index().plot(x = 'nombre_enfants', kind = 'bar')

# Evaluate the financial impact of a reform

Write a parametric reform tthat increseases the top marginal tax rates and evaluate how much revenue can be collected

In [ ]:
year = 2017
def modify_parameters(parameters):
    parameters.prelevements_obligatoires.impots_directs.bareme_impot_progressif[5].rate.update(period = period(year), value = .5)
    return parameters
    
class tax_the_rich(Reform):
    name = u"Tax last bracket at 50%"

    def apply(self):
        self.modify_parameters(modifier_function = modify_parameters)

In [ ]:
senegal_tax_benefit_system = SenegalTaxBenefitSystem()
tax_the_rich_tax_benefit_system = tax_the_rich(senegal_tax_benefit_system)

In [ ]:
scenario = SenegalSurveyScenario(
    data = data,
    tax_benefit_system = tax_the_rich_tax_benefit_system,
    baseline_tax_benefit_system = senegal_tax_benefit_system,
    year = 2017,
    )

In [ ]:
print('reform tax the rich: ', scenario.compute_aggregate('impot_revenus', period = 2017) / 1e9)
print('baseline: ', scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True) / 1e9)

In [ ]:
from openfisca_senegal.entities import Person

def build_ultimate_reform_tax_benefit_system(threshold = 0, marginal_tax_rate = .4):
    year = 2017

    senegal_tax_benefit_system = SenegalTaxBenefitSystem()
    class impot_revenus(Variable):
        def formula(person, period):
            impot_avant_reduction_famille = person('impot_avant_reduction_famille', period)
            reduction_impots_pour_charge_famille = person('reduction_impots_pour_charge_famille', period)
            impot_apres_reduction_famille = impot_avant_reduction_famille - reduction_impots_pour_charge_famille
            impot_revenus = max_(0, impot_apres_reduction_famille)
            return impot_revenus * (impot_revenus > threshold)            

    def modify_parameters(parameters):
        parameters.prelevements_obligatoires.impots_directs.bareme_impot_progressif[5].rate.update(period = period(year), value = marginal_tax_rate)
        return parameters


    class ultimate_reform(Reform):
        name = u"Tax the rich and save the poor taxpayers (tax < {})".format(threshold)

        def apply(self):
            self.update_variable(impot_revenus)
            self.modify_parameters(modifier_function = modify_parameters)

    return ultimate_reform(senegal_tax_benefit_system)

In [ ]:
reformed_tax_benefit_system = build_ultimate_reform_tax_benefit_system(threshold = 100000, marginal_tax_rate = .45)

In [ ]:
scenario = SenegalSurveyScenario(
    data = data,
    tax_benefit_system = reformed_tax_benefit_system,
    baseline_tax_benefit_system = SenegalTaxBenefitSystem(),
    year = 2017
    )

In [ ]:
print('reform: ', scenario.compute_aggregate('impot_revenus', period = 2017) / 1e9)
print('baseline: ', scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True) / 1e9)

In [ ]:
scenario = SenegalSurveyScenario(
    data = data,
    tax_benefit_system = reformed_tax_benefit_system,
    baseline_tax_benefit_system =  SenegalTaxBenefitSystem(),
    year = 2017
    )
print(scenario.compute_aggregate('impot_revenus', period = 2017) / 1e9)
print(scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True) / 1e9)
cost = - (
    scenario.compute_aggregate('impot_revenus', period = 2017) -
    scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True)
    ) / 1e9
print(cost)

In [ ]:
def compute_reform_cost(threshold = 0, marginal_tax_rate = .4):
    reformed_tax_benefit_system = build_ultimate_reform_tax_benefit_system(
        threshold = threshold, 
        marginal_tax_rate = float(marginal_tax_rate)  
        # We need to convert to float here since fsolve use numpy array which are not accepted as 
        # legislation parameters
        )
    scenario = SenegalSurveyScenario(
        data = data,
        tax_benefit_system = reformed_tax_benefit_system,
        baseline_tax_benefit_system = SenegalTaxBenefitSystem(),
        year = 2017
        )
    cost = - (
        scenario.compute_aggregate('impot_revenus', period = 2017) - 
        scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True)
        ) / 1e9
    return cost

In [ ]:
def compute_reform_cost_from_marginal_tax_rate(marginal_tax_rate = .4):
    return compute_reform_cost(threshold = 100000, marginal_tax_rate = marginal_tax_rate)

def compute_reform_cost_from_threshold(threshold = 0):
    return compute_reform_cost(threshold = threshold, marginal_tax_rate = .41)


In [ ]:
from scipy.optimize import fsolve
balancing_marginal_tax_rate = fsolve(compute_reform_cost_from_marginal_tax_rate, .40)
balancing_threshold = fsolve(compute_reform_cost_from_threshold, 1000)
print("balancing_marginal_tax_rate to finance a 100000 threshold: {}".format(float(balancing_marginal_tax_rate)))
print("balancing_threshold financed by an increase from 40% to 41%: {}".format(float(balancing_threshold)))

In [ ]:
print(compute_reform_cost_from_marginal_tax_rate(marginal_tax_rate = balancing_marginal_tax_rate))
print(compute_reform_cost_from_threshold(threshold = balancing_threshold))


In [ ]:
reformed_tax_benefit_system = build_ultimate_reform_tax_benefit_system(
    threshold = balancing_threshold, 
    marginal_tax_rate = .41
    )
scenario = SenegalSurveyScenario(
    data = data,
    tax_benefit_system = reformed_tax_benefit_system,
    baseline_tax_benefit_system = SenegalTaxBenefitSystem(),
    year = 2017
    )

In [ ]:
scenario.compute_pivot_table(
    aggfunc = 'sum', 
    values = ['impot_revenus'], 
    columns = ['nombre_enfants'],
    period = 2017,
    difference = True,
    ).stack().reset_index().plot(x = 'nombre_enfants', kind = 'bar')

In [ ]:
scenario.compute_pivot_table(
    aggfunc = 'sum', 
    values = ['impot_revenus'], 
    columns = ['nombre_enfants'],
    period = 2017,
    difference = True,
    ).stack().reset_index()

In [ ]:
scenario.compute_pivot_table(
    aggfunc = 'sum', 
    values = ['impot_revenus'], 
    columns = ['nombre_enfants'],
    period = 2017,
    difference = True,
    ).stack().sum() / 1e9
# The result is close to zéro and almost equal to the one computed above